In [ ]:
import numpy as np
import pygetm

%matplotlib widget
import matplotlib.pyplot

In [ ]:
Lx, Ly = 100.0, 100.0
nx, ny = 100, 101
x = np.linspace(-Lx / 2, Lx / 2, nx)
y = np.linspace(-Ly / 2, Ly / 2, ny)
domain = pygetm.domain.create_cartesian(x, y, nz=1, H=1.0, lat=0.0)
domain.initialize(runtype=pygetm.BAROTROPIC_2D)

In [ ]:
period = 600
omega = 2 * np.pi / period
cfl = 1.0
umax = omega * Lx / 2
dt_cfl = cfl * min(Lx / (nx + 1), Ly / (ny + 1)) / umax
no_of_revolutions = 5
Nmax = no_of_revolutions * round(2 * np.pi / omega / dt_cfl)
tmax = no_of_revolutions * 2 * np.pi / omega
timestep = tmax / Nmax

In [ ]:
u = domain.U.array(fill=0.0)
v = domain.V.array(fill=0.0)
var = domain.T.array(fill=1.0)
u[:, :] = -omega * domain.U.y
v[:, :] = omega * domain.V.x
u[(2 * domain.U.x / Lx) ** 2 + (2 * domain.U.y / Ly) ** 2 > 1] = 0.0
v[(2 * domain.V.x / Lx) ** 2 + (2 * domain.V.y / Ly) ** 2 > 1] = 0.0
var[int(0.2 * ny) : int(0.4 * ny), int(0.2 * nx) : int(0.4 * nx),] = 5.0

In [ ]:
fig, ax = matplotlib.pyplot.subplots()
u_destag, v_destag = u.interp(domain.T), v.interp(domain.T)
pc = ax.quiver(u_destag[::5, ::5], v_destag[::5, ::5])

In [ ]:
nsample = 4
fig, ax = matplotlib.pyplot.subplots()
pc = ax.pcolormesh(var)
cb = fig.colorbar(pc)
title = ax.set_title("time step: 0 of %i" % Nmax)

In [ ]:
totals = [var[...].sum()]

apply_horizontal_diffusion = False

if apply_horizontal_diffusion:
    Ah = domain.T.array(fill_value=0.01)
    
advect = pygetm.operators.Advection(domain.T, scheme=1)
for i in range(1, Nmax):
    if apply_horizontal_diffusion:
        advect(u, v, timestep, var, Ah=Ah)
    else:
        advect(u, v, timestep, var)     
    if i % nsample == 0:
        pc.set_array(var[...].ravel())
        title.set_text("time step: %i of %i" % (i, Nmax))
        fig.canvas.draw()
    totals.append(var[...].sum())

In [ ]:
fig, ax = matplotlib.pyplot.subplots()
ax.plot(totals / totals[0] - 1.0)
ax.grid()
ax.set_xlabel("time step")
ax.set_ylabel("change in total tracer")
fig.show()